Search Engine with tools and agents

In [2]:
##Tools

from langchain.tools import WikipediaQueryRun,ArxivQueryRun
from langchain_community.utilities import WikipediaAPIWrapper,ArxivAPIWrapper

ModuleNotFoundError: No module named 'langchain'

In [2]:
##inbuilt tool of wikipedia

wiki_api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=250)
wikipedia=WikipediaQueryRun(api_wrapper=wiki_api_wrapper)

In [3]:
arxiv_api_wrapper=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=250)
arxiv=ArxivQueryRun(api_wrapper=arxiv_api_wrapper)
arxiv.name

tools=[wikipedia,arxiv]

In [4]:
##custom tools

from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools.retriever import create_retriever_tool
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader

import os
from dotenv import load_dotenv
load_dotenv()

os.environ["HF_TOKEN"]=os.getenv("HF_TOKEN")
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

/home/himanshu-singh/Desktop/Langchain/venv/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3577: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)
USER_AGENT environment variable not set, consider setting it to identify your requests.
/home/himanshu-singh/Desktop/Langchain/venv/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://

In [5]:
loader=WebBaseLoader("https://python.langchain.com/docs/introduction/")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb=Chroma.from_documents(documents,embedding=embeddings)
retriever=vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x775825f7d1c0>, search_kwargs={})

In [6]:
retriever_tool=create_retriever_tool(retriever,"langsmith_search","Search any information about langsmith")

In [7]:
tools=[arxiv,wikipedia,retriever_tool]

In [8]:
##run all this tools with agents and LLM models
from langchain_groq import ChatGroq

api_key=os.getenv("GROQ_API_KEY")

llm=ChatGroq(model="llama3-8b-8192",groq_api_key=api_key)


In [9]:
##prompt template

from langchain import hub

prompt=hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [10]:
##agent creation 
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm,tools,prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [11]:
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [12]:
agent_executor.invoke({"input":"whom did i ask about"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'whom did i ask about'}`


Page: 10 Things I Hate About You
Summary: 10 Things I Hate About You is a 1999 American teen romantic comedy film directed by Gil Junger in his film directorial debut and starring Julia Stiles, Heath Ledger, Joseph Gordon-Levitt, and Larisa Oleynik. 
Invoking: `arxiv` with `{'query': '10 Things I Hate About You'}`


Published: 2009-07-10
Title: Fermat Reals - Nilpotent Infinitesimals and Infinite Dimensional Spaces
Authors: Paolo Giordano
Summary: F.: Good morning Hermann, I would like to talk with you about infinitesimals.
  G.: Tell me Pierre.
  F.: I'm fed upUnfortunately, it seems that the result of the tool call "call_ktba" is not relevant to the original question "whom did I ask about". The question was about the person or entity you asked about, but the result is about a mathematics paper.

Let's try another approach. The original result from the tool call "call_8ye3" mentio

{'input': 'whom did i ask about',
 'output': 'Unfortunately, it seems that the result of the tool call "call_ktba" is not relevant to the original question "whom did I ask about". The question was about the person or entity you asked about, but the result is about a mathematics paper.\n\nLet\'s try another approach. The original result from the tool call "call_8ye3" mentioned a movie "10 Things I Hate About You". Could you please provide more context or clarify what you meant by "whom did I ask about"?'}